In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-29339a8f-8a05-4d10-9a0a-a4718f79a53d",
    secret_key="sk-lf-d1f94511-893b-456e-973e-f57b44d50a30",
    host="https://cloud.langfuse.com"
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

In [4]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [5]:
# load documents
documents = SimpleDirectoryReader(input_files=["paul_graham_essay.txt"]).load_data()

In [8]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [21]:


response1 = query_engine.query("what did paul graham do after going to RISD")
display(Markdown(f"<b>{response1}</b>"))

<b>After going to RISD, Paul Graham started painting still lives in his bedroom at night.</b>

In [19]:
for node in response1.source_nodes:
    print(node.metadata)
    print(node.get_content()[:50])

{'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-03', 'last_modified_date': '2024-04-03'}
I didn't want to drop out of grad school, but how 
{'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-03', 'last_modified_date': '2024-04-03'}
What I Worked On

February 2021

Before college th


#### Now, we use HyDEQueryTransform to generate a hypothetical document and use it for embedding lookup.

In [9]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query("what did paul graham do after going to RISD")
display(Markdown(f"{response}"))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


After going to RISD, Paul Graham wrote lots of essays on various topics, had dinners for a group of friends every Thursday night, worked on spam filters, did some painting, and bought another building in Cambridge to use as an office. He also attended a party at his house in October 2003, where he met Jessica Livingston, who later became his partner in starting an investment firm.

In [10]:
for node in response.source_nodes:
    print(node.metadata)
    print(node.get_content()[:50])

{'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-03', 'last_modified_date': '2024-04-03'}
In return for that and doing the initial legal wor
{'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75042, 'creation_date': '2024-04-03', 'last_modified_date': '2024-04-03'}
Over the next several years I wrote lots of essays


In [10]:
query_bundle = hyde("what did paul graham do after going to RISD")
hyde_doc = query_bundle.embedding_strs[0]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
hyde_doc

'After attending the Rhode Island School of Design (RISD), Paul Graham went on to co-found Viaweb, an online store builder that was later acquired by Yahoo for $49 million. Following this success, Graham became an influential figure in the tech industry, co-founding the startup accelerator Y Combinator in 2005. Y Combinator has since become one of the most prestigious and successful startup accelerators in the world, helping launch companies like Dropbox, Airbnb, and Reddit. In addition to his work with Y Combinator, Graham is also a prolific writer and essayist, known for his insightful and thought-provoking writings on startups, technology, and entrepreneurship. Overall, Paul Graham\'s career after RISD has been marked by innovation, success, and a lasting impact on the tech industry."""'